# 1. Configuration

### Imports

In [1]:
import pygad
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 

### Read the data

In [2]:
dataset = pd.read_csv("dataset.csv")
dataset

,Equation,Xs,Ys
0,((x ** 4) - 6),"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[-5, 10, 75, 250, 619, 1290, 2395, 4090, 6555,..."
1,(((x / 8) * 2) + 1),"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0, 3..."
2,(((x - 1) - 3) / 5),"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[-0.6, -0.4, -0.2, 0.0, 0.2, 0.4, 0.6, 0.8, 1...."
3,(x * 5),"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60..."
4,(x + 2),"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ..."
...,...,...,...
93,1*x**5 + -2*x**3 + -1*x + -5,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[-7, 9, 181, 887, 2865, 7333, 16109, 31731, 57..."
94,3*x**5 + 2*x**4 + 1*x + -2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[4, 128, 892, 3586, 10628, 25924, 55228, 10650..."
95,-4*x**5 + -1*x + -5,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[-10, -135, -980, -4105, -12510, -31115, -6724..."
96,4*x + +1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[5, 9, 13, 17, 21, 25, 29, 33, 37, 41, 45, 49,..."


# 2. Representation

### Tree data structure

#### Node class

In [3]:
class Node:
    def __init__(self, value=None):
        self.value = value
        self.children = []

    def add_child(self, child):
        self.children.append(child)

#### Equation tree class

In [4]:
class EquationTree:
    def __init__(self, root=None):
        self.root = root

    def build_tree(self, expression):
        self.root = self._build_tree_recursive(expression)

    def _build_tree_recursive(self, expression):
        
        current_depth = 0

        lowest_depth = 0
        lowest_depth_index = -1

        leaf = False
        leaf_value = None


        # ** nima prednosti pred * in / in je treba to popraut, oz nevem c ma + in - prednost pred * in /
        for i in range(len(expression)):
            if expression[i] == '(':
                current_depth += 1
            elif expression[i] == ')':
                current_depth -= 1
            elif expression[i] in ['+', '-', '*', '/','**']:
                if current_depth <= lowest_depth or lowest_depth_index == -1:
                    lowest_depth = current_depth
                    lowest_depth_index = i
            elif expression[i] == 'x' or expression[i].lstrip('-').isdigit():
                leaf_value = expression[i]
                leaf = True

        if lowest_depth_index == -1 and leaf:
            return Node(leaf_value)
        else:
            root = Node(expression[lowest_depth_index])
            root.add_child(self._build_tree_recursive(expression[:lowest_depth_index]))
            root.add_child(self._build_tree_recursive(expression[lowest_depth_index+1:]))

            return root
        
    def print_tree(self):
        self._print_tree_recursive(self.root)
    
    def _print_tree_recursive(self, root, depth=0):
        if root is None:
            return

        print('\t' * depth + str(root.value))

        for child in root.children:
            self._print_tree_recursive(child, depth + 1)

    def evaluate(self, x):
        return self._evaluate_recursive(self.root, x)
    
    def _evaluate_recursive(self, root, x):
        if root is None:
            return None

        if root.value == 'x':
            return x
        elif root.value.lstrip('-').isdigit():
            return int(root.value)
        elif root.value == '+':
            return self._evaluate_recursive(root.children[0], x) + self._evaluate_recursive(root.children[1], x)
        elif root.value == '-':
            return self._evaluate_recursive(root.children[0], x) - self._evaluate_recursive(root.children[1], x)
        elif root.value == '*':
            return self._evaluate_recursive(root.children[0], x) * self._evaluate_recursive(root.children[1], x)
        elif root.value == '/':
            return self._evaluate_recursive(root.children[0], x) / self._evaluate_recursive(root.children[1], x)
        elif root.value == '**':
            return self._evaluate_recursive(root.children[0], x) ** self._evaluate_recursive(root.children[1], x)
        else:
            return None

In [5]:
equation_inputs = dataset.iloc[0].values[0]
equation_inputs

'((x ** 4) - 6)'

In [6]:
def stringEQtoArray(equation):
    arr = []
    skip = False

    
    for index, char in enumerate(equation):
        if skip:
            skip = False
            continue
    
        if char == ' ':
            continue
        elif char == 'x' or char.isdigit():
            arr.append(char)
        elif char == '*' and equation[index+1] == '*':
            arr.append('**')
            skip = True
        elif char in ['+', '-'] and equation[index+1].isdigit():
            if char == '+':
                arr.append(equation[index+1])
            else:
                arr.append(char + equation[index+1])
            skip = True
        elif char in ['+', '-', '*', '/', '(', ')']:
            arr.append(char)

    return arr

        
equation = stringEQtoArray(equation_inputs)
equation

['(', '(', 'x', '**', '4', ')', '-', '6', ')']

In [7]:
equation_tree = EquationTree()
equation_tree.build_tree(equation) 
equation_tree.print_tree()
equation_tree.evaluate(2)

-
	**
		x
		4
	6


10